# Net Exposure + PnL Tracker (Underlying ↔ Levered ETFs)

This notebook aggregates **net underlying exposure** by mapping each ETF position into *underlying-equivalent shares*:

\[
\text{Underlying Eq Shares} = \text{ETF Shares} \times \text{Leverage}
\]

It also groups **PnL by underlying**, summing PnL across the underlying stock (if held) plus its associated ETFs.


In [23]:
from pathlib import Path
import pandas as pd
import numpy as np

# --- Inputs (edit if your paths differ) ---
SCREENED_PATH = Path("data/etf_screened_today.csv")  # leverage map & ETF↔underlying map
# Fallbacks if you run this from a different working directory:
FALLBACK_SCREENED = Path("/data/etf_screened_today.csv")

BASE_DIR = Path.cwd().parent          # notebooks/ -> project root
DATA_DIR = BASE_DIR / "data"
# Portfolio positions export (choose ONE source that matches your workflow)
PORT_PATHS = [
    Path(DATA_DIR / "Levered ETF Autotrading.csv")
]

def _first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    return None

screened_path = Path(DATA_DIR / "etf_screened_today.csv")

port_path = _first_existing(PORT_PATHS)
print("Using leverage/map file:", screened_path)
print("Using positions file:", port_path if port_path else "(none found yet)")

Using leverage/map file: c:\Users\werdn\Documents\Investing\ls-algo\data\etf_screened_today.csv
Using positions file: c:\Users\werdn\Documents\Investing\ls-algo\data\Levered ETF Autotrading.csv


## 1) Load ETF→Underlying + Leverage map

`etf_screened_today.csv` should contain at least:
- `ETF`
- `Underlying`
- `Leverage`


In [24]:
screened = pd.read_csv(screened_path)

required = {"ETF", "Underlying", "Leverage"}
missing = required - set(screened.columns)
if missing:
    raise ValueError(f"etf_screened_today.csv missing required columns: {sorted(missing)}")

# clean
screened["ETF"] = screened["ETF"].astype(str).str.strip()
screened["Underlying"] = screened["Underlying"].astype(str).str.strip()
screened["Leverage"] = pd.to_numeric(screened["Leverage"], errors="coerce")
screened = screened.dropna(subset=["ETF", "Underlying", "Leverage"]).copy()

etf_to_under = dict(zip(screened["ETF"], screened["Underlying"]))
etf_to_lev = dict(zip(screened["ETF"], screened["Leverage"]))

screened.head()

,ETF,Underlying,Leverage,cagr_port_hist,LevType,borrow_current,shares_available,borrow_spiking,cagr_positive,borrow_leq_cap,borrow_gt_cap,whitelisted,exclude_borrow_gt_cap,exclude_no_shares,exclude_borrow_spike,include_for_algo
0,AAPU,AAPL,2.0,0.026743,2x,0.048070,5000,False,True,True,False,False,False,False,False,True
1,AMUU,AMD,2.0,0.057246,2x,0.000000,40000,False,True,True,False,False,False,False,False,True
2,AMZU,AMZN,2.0,0.050622,2x,0.062528,200000,False,True,True,False,False,False,False,False,True
3,AVL,AVGO,2.0,0.039395,2x,0.035776,250000,False,True,True,False,False,False,False,False,True
4,BITX,IBIT,2.0,0.071068,2x,0.082580,600000,False,True,True,False,False,False,False,False,True


## 2) Load positions

This notebook is flexible, but it needs at least:

- `symbol` (or `Symbol`)
- `qty` (position size; **short negative**)

Optional but recommended for PnL:
- `avg_cost` (or `avgCost`, `average_cost`)
- `last` (or `price`, `market_price`)
- `unrealized_pnl` (if you already compute it)
- `realized_pnl` (if available)

If your columns differ, edit the `colmap` section.


In [88]:
import numpy as np
import pandas as pd
import re
from pathlib import Path

# ---------- helpers ----------
import numpy as np
import pandas as pd
import re

def parse_number_series(s: pd.Series) -> pd.Series:
    """
    Broker-export safe numeric parser.
    Handles:
      - commas
      - $ signs / text like 'Shares'
      - (123) -> -123
      - blanks / '--' -> NaN
      - non-breaking spaces
    """
    x = s.astype(str)

    # normalize whitespace (including NBSP)
    x = x.str.replace("\u00a0", " ", regex=False).str.strip()

    # treat common null tokens as NaN
    x = x.replace({"": np.nan, "—": np.nan, "–": np.nan, "--": np.nan, "N/A": np.nan, "NA": np.nan})

    # parentheses negatives: (123.45) -> -123.45
    x = x.str.replace(r"^\((.*)\)$", r"-\1", regex=True)

    # remove commas
    x = x.str.replace(",", "", regex=False)

    # remove ANYTHING that's not digit, minus, or dot
    x = x.str.replace(r"[^0-9\.\-]", "", regex=True)

    # collapse multiple minus signs (rare formatting glitches)
    x = x.str.replace(r"^-{2,}", "-", regex=True)

    # empty after stripping -> NaN
    x = x.replace("", np.nan)

    return pd.to_numeric(x, errors="coerce")

def clean_symbol(s: pd.Series) -> pd.Series:
    """
    Clean ticker strings from broker exports.
    Normalizes:
      - trims
      - removes trailing "(EXCH)" if present
      - uppercases
      - converts BRK B / BRK.B -> BRK-B
    """
    x = s.astype(str).str.strip()

    # drop "(NASDAQ)" etc if present
    x = x.str.replace(r"\s*\(.*?\)\s*$", "", regex=True)

    x = s.str.upper()

    # normalize share-class punctuation/spacing: BRK B or BRK.B -> BRK-B
    x = x.str.replace(".", "-", regex=False)
    x = x.str.replace(r"\s+", "-", regex=True)

    return x

# ---------- main loader ----------
def load_positions_from_tws_export(port_path: str | Path) -> pd.DataFrame:
    port_path = Path(port_path)
    if not port_path.exists():
        raise FileNotFoundError(f"Positions file not found: {port_path}")

    pos = pd.read_csv(port_path)
    # Hard-pick these based on your confirmed export
    sym_c = "Financial Instrument"
    qty_c = "Position"

    if sym_c not in pos.columns:
        raise ValueError(f"Expected column '{sym_c}' not found. Columns: {pos.columns.tolist()[:60]}")
    if qty_c not in pos.columns:
        raise ValueError(f"Expected column '{qty_c}' not found. Columns: {pos.columns.tolist()[:60]}")

    df = pos.copy()
    # --- symbol ---
    df["symbol_raw"] = df[sym_c]
    df["symbol"] = clean_symbol(df[sym_c])

    # --- quantity ---
    df["qty"]  =parse_number_series(df[qty_c]).fillna(0.0)
    # Optional columns if present
    if "Avg Price" in df.columns:
        df["avg_cost"] = parse_number_series(df["Avg Price"])
    else:
        df["avg_cost"] = np.nan

    if "Last" in df.columns:
        df["last"] = parse_number_series(df["Last"])
    else:
        df["last"] = np.nan

    if "Unrealized P&L" in df.columns:
        df["unrealized_pnl"] = parse_number_series(df["Unrealized P&L"])
    else:
        df["unrealized_pnl"] = np.nan

    # Realized usually not in this export
    df["realized_pnl"] = 0.0

    # Drop zero positions
    df = df[df["qty"].abs() > 0].copy()

    # Final sanity checks
    df["symbol"] = df["symbol"].astype(str).str.strip()
    df = df[df["symbol"].ne("")].copy()

    # Keep a tidy output schema
    keep = ["symbol", "qty", "avg_cost", "last", "unrealized_pnl", "realized_pnl", "symbol_raw"]
    keep = [c for c in keep if c in df.columns]
    df = df[keep].reset_index(drop=True)

    return df

# --- usage ---
df = load_positions_from_tws_export(port_path)
df


,symbol,qty,avg_cost,last,unrealized_pnl,realized_pnl,symbol_raw
0,AAPU,-38.0,31.548,30.63,35.0,0.0,AAPU
1,ABNB,13.0,134.565,132.78,-23.0,0.0,ABNB
2,AAPL,25.0,259.991,260.24,4.0,0.0,AAPL
3,ABNY,-36.0,46.384,47.69,26.0,0.0,ABNY
4,AI,129.0,13.615,13.90,38.0,0.0,AI
...,...,...,...,...,...,...,...
154,XXRP,-90.0,11.487,12.66,-105.0,0.0,XXRP
155,XYLD,-42.0,40.856,41.00,-6.0,0.0,XYLD
156,XYLG,-63.0,27.654,27.74,-7.0,0.0,XYLG
157,XYZ,26.0,66.374,65.72,-9.0,0.0,XYZ


## 3) Classify rows: Underlyings vs ETFs

- If `symbol` exists in `etf_to_under`, treat it as an ETF
- Otherwise, treat it as an underlying stock (direct holding)


In [90]:
import numpy as np
import pandas as pd
import re

def norm_sym(x: str) -> str:
    """Normalize symbols so TWS exports match your mapping keys."""
    if pd.isna(x):
        return ""
    s = str(x).upper().strip()

    # Common normalizations
    s = re.sub(r"\s+", " ", s)        # collapse whitespace
    s = s.replace("/", "-")           # just in case
    s = s.replace(".", "-")           # BRK.B -> BRK-B
    s = s.replace(" ", "-")           # BRK B -> BRK-B

    return s
df["symbol_raw"] = df["symbol"].astype(str)

# take first token before space as ticker
df["symbol"] = df["symbol_raw"].str.upper().str.strip().str.split().str[0]

# --- Build normalized maps (IMPORTANT) ---
etf_to_under_norm = {norm_sym(k): norm_sym(v) for k, v in etf_to_under.items()}
etf_to_lev_norm   = {norm_sym(k): float(v) for k, v in etf_to_lev.items()}

# --- Normalize your df symbols once ---
df["symbol_raw"] = df["symbol"]
df["sym_key"] = df["symbol"].apply(norm_sym)

# --- Mapping ---
df["is_etf"] = df["sym_key"].isin(etf_to_under_norm.keys())

df["underlying"] = np.where(
    df["is_etf"],
    df["sym_key"].map(etf_to_under_norm),
    df["sym_key"],   # stock/fallback = itself (normalized)
)

# leverage: if ETF but leverage missing, default to 1.0 (or raise—your call)
df["leverage"] = np.where(
    df["is_etf"],
    df["sym_key"].map(etf_to_lev_norm).astype(float),
    1.0
)

# Fill any mapping misses defensively
df["underlying"] = df["underlying"].fillna(df["sym_key"])
df["leverage"] = pd.to_numeric(df["leverage"], errors="coerce").fillna(1.0)

# Underlying-equivalent shares contributed by this row
df["under_eq_shares"] = df["qty"] * df["leverage"]

df[["symbol_raw","sym_key","qty","is_etf","underlying","leverage","under_eq_shares"]]


,symbol_raw,sym_key,qty,is_etf,underlying,leverage,under_eq_shares
0,AAPU,AAPU,-38.0,True,AAPL,2.0,-76.0
1,ABNB,ABNB,13.0,False,ABNB,1.0,13.0
2,AAPL,AAPL,25.0,False,AAPL,1.0,25.0
3,ABNY,ABNY,-36.0,True,ABNB,1.0,-36.0
4,AI,AI,129.0,False,AI,1.0,129.0
...,...,...,...,...,...,...,...
154,XXRP,XXRP,-90.0,True,GXRP,2.0,-180.0
155,XYLD,XYLD,-42.0,True,SPY,1.0,-42.0
156,XYLG,XYLG,-63.0,True,SPY,1.0,-63.0
157,XYZ,XYZ,26.0,False,XYZ,1.0,26.0


In [ ]:
df["is_etf"] = df["symbol"].isin(etf_to_under)
df["underlying"] = np.where(df["is_etf"], df["symbol"].map(etf_to_under), df["symbol"])
df["leverage"] = np.where(df["is_etf"], df["symbol"].map(etf_to_lev), 1.0)

# Underlying-equivalent shares contributed by this row
df["under_eq_shares"] = df["qty"] * df["leverage"]

df[["symbol","qty","is_etf","underlying","leverage","under_eq_shares"]]

,symbol,qty,is_etf,underlying,leverage,under_eq_shares
0,AAPU,-38.0,True,AAPL,2.0,-76.0
1,ABNB,13.0,False,ABNB,1.0,13.0
2,AAPL,25.0,False,AAPL,1.0,25.0
3,ABNY,-36.0,True,ABNB,1.0,-36.0
4,AI,129.0,False,AI,1.0,129.0
...,...,...,...,...,...,...
154,XXRP,-90.0,True,GXRP,2.0,-180.0
155,XYLD,-42.0,True,SPY,1.0,-42.0
156,XYLG,-63.0,True,SPY,1.0,-63.0
157,XYZ,26.0,False,XYZ,1.0,26.0


## 4) Compute PnL per position (if not already provided)

If `unrealized_pnl` is missing but `avg_cost` and `last` exist, we compute:

\[
UPnL = (last - avg\_cost) \times qty
\]

Total PnL = realized + unrealized


In [92]:
# Fill unrealized_pnl if missing and we have avg_cost & last
need_calc = df["unrealized_pnl"].isna() & df["avg_cost"].notna() & df["last"].notna()
df.loc[need_calc, "unrealized_pnl"] = (df.loc[need_calc, "last"] - df.loc[need_calc, "avg_cost"]) * df.loc[need_calc, "qty"]

# If still missing, keep as 0 for aggregation (but you may want to investigate)
df["unrealized_pnl"] = df["unrealized_pnl"].fillna(0.0)
df["realized_pnl"] = df["realized_pnl"].fillna(0.0)

df["total_pnl"] = df["unrealized_pnl"] + df["realized_pnl"]

df[["symbol","qty","avg_cost","last","unrealized_pnl","realized_pnl","total_pnl"]].head(20)

,symbol,qty,avg_cost,last,unrealized_pnl,realized_pnl,total_pnl
0,AAPU,-38.0,31.548,30.63,35.0,0.0,35.0
1,ABNB,13.0,134.565,132.78,-23.0,0.0,-23.0
2,AAPL,25.0,259.991,260.24,4.0,0.0,4.0
3,ABNY,-36.0,46.384,47.69,26.0,0.0,26.0
4,AI,129.0,13.615,13.90,38.0,0.0,38.0
5,AIYY,-101.0,17.324,17.55,-23.0,0.0,-23.0
6,ALAB,13.0,166.798,172.14,71.0,0.0,71.0
7,AMD,21.0,216.773,223.71,147.0,0.0,147.0
8,AMDG,-24.0,25.866,28.36,-60.0,0.0,-60.0
9,AMUU,-20.0,55.687,58.80,-61.0,0.0,-61.0


## 5) Net exposure per underlying (lever-adjusted)

This is the key view:
- Sum of `under_eq_shares` for all rows mapped to the same underlying.
- Positive = net long underlying-equivalent
- Negative = net short underlying-equivalent


In [107]:
# --- per-leg notional (signed) ---
# assumes:
#   df["qty"]  = Position (signed)
#   df["last"] = Last price (positive)
df = df.copy()

df["mv"] = df["under_eq_shares"] * df["last"]          # signed market value (long +, short -)
df["gross_mv"] = df["mv"].abs()            # gross notional per leg

exposure_by_under = (
    df.groupby("underlying", as_index=False)
      .agg(
          net_notional_usd=("mv", "sum"),
          gross_notional_usd=("gross_mv", "sum"),
          n_legs=("sym_key","nunique"),
          legs=("sym_key", lambda s: ",".join(sorted(set(s))[:20]))
      )
      .sort_values("net_notional_usd", ascending=False)
)

exposure_by_under


,underlying,net_notional_usd,gross_notional_usd,n_legs,legs
42,NVDA,4703.47,22061.25,5,"NVDA,NVDG,NVDO,NVDU,NVDY"
5,AMZN,4694.71,16126.09,4,"AMZN,AMZU,AMZW,AMZY"
31,IBIT,4689.64,13911.32,3,"BITX,BTCL,IBIT"
45,PLTR,4677.13,12814.89,3,"PLTR,PLTU,PLTW"
37,META,3731.01,13516.99,4,"FBY,META,METU,METW"
...,...,...,...,...,...
11,BA,-1362.44,6214.44,3,"BA,BOEG,BOEU"
39,MSTR,-1448.77,9330.93,5,"MSTR,MSTU,MSTW,MSTX,MSTY"
28,GOOGL,-1513.75,7560.85,4,"GGLL,GOOGL,GOOX,GOOY"
20,COIN,-3976.57,3976.57,3,"COIG,COIW,CONY"


## 6) PnL by underlying (group of ETFs + underlying)

This groups:
- underlying stock position (if held)
- all associated ETF legs

So you can see which underlying buckets are making/losing money.


In [99]:
EXCLUDE_SYMBOLS = {"BRK-B"}

df = df[~df["symbol"].isin(EXCLUDE_SYMBOLS)].copy()

In [100]:
pnl_by_under = (
    df.groupby("underlying", as_index=False)
      .agg(
          unrealized_pnl=("unrealized_pnl","sum"),
          realized_pnl=("realized_pnl","sum"),
          total_pnl=("total_pnl","sum"),
          net_under_eq_shares=("under_eq_shares","sum"),
      )
      .sort_values("total_pnl", ascending=False)
)

pnl_by_under.head(30)

,underlying,unrealized_pnl,realized_pnl,total_pnl,net_under_eq_shares
41,NFLX,161.0,0.0,161.0,-296.0
55,TSLA,144.0,0.0,144.0,-259.0
37,META,118.0,0.0,118.0,-243.0
36,MARA,109.0,0.0,109.0,159.0
38,MSFT,105.0,0.0,105.0,-299.0
42,NVDA,85.0,0.0,85.0,-338.0
45,PLTR,73.0,0.0,73.0,-30.0
0,AAPL,60.0,0.0,60.0,-189.0
49,RBLX,40.0,0.0,40.0,-39.0
7,ARM,30.0,0.0,30.0,-54.0


## 7) Drilldown: show the ETF/underlying legs for a chosen underlying

Edit `NAME` and rerun to see the component legs (symbols, qty, leverage, pnl).


In [101]:
NAME = "AAPL"  # <-- change me

drill = df[df["underlying"] == NAME].copy()
drill = drill.sort_values(["is_etf","symbol"])
drill[["symbol","qty","leverage","under_eq_shares","unrealized_pnl","realized_pnl","total_pnl"]]

,symbol,qty,leverage,under_eq_shares,unrealized_pnl,realized_pnl,total_pnl
2,AAPL,25.0,1.0,25.0,4.0,0.0,4.0
0,AAPU,-38.0,2.0,-76.0,35.0,0.0,35.0
14,APLY,-138.0,1.0,-138.0,21.0,0.0,21.0


## 8) Portfolio totals + exports

Writes two CSVs you can use for dashboards / daily emails:
- `net_exposure_by_underlying.csv`
- `pnl_by_underlying.csv`


In [102]:
totals = {
    "net_under_eq_shares": float(df["under_eq_shares"].sum()),
    "gross_under_eq_shares": float(np.abs(df["under_eq_shares"]).sum()),
    "total_unrealized_pnl": float(df["unrealized_pnl"].sum()),
    "total_realized_pnl": float(df["realized_pnl"].sum()),
    "total_pnl": float(df["total_pnl"].sum()),
}
totals

{'net_under_eq_shares': -6027.0,
 'gross_under_eq_shares': 10943.0,
 'total_unrealized_pnl': 384.0,
 'total_realized_pnl': 0.0,
 'total_pnl': 384.0}

In [103]:
out_dir = Path("data") / "reports"
out_dir.mkdir(parents=True, exist_ok=True)

exposure_csv = out_dir / "net_exposure_by_underlying.csv"
pnl_csv = out_dir / "pnl_by_underlying.csv"

exposure_by_under.to_csv(exposure_csv, index=False)
pnl_by_under.to_csv(pnl_csv, index=False)

print("Wrote:", exposure_csv)
print("Wrote:", pnl_csv)

Wrote: data\reports\net_exposure_by_underlying.csv
Wrote: data\reports\pnl_by_underlying.csv
